<a href="https://colab.research.google.com/github/Lukaid/my_notebook/blob/main/blog_scraping_using_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import pandas as pd

def get_posts(url):

    search_url = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(search_url, 'html.parser')
    blog_url = 'https://blog.naver.com' + soup.select('iframe#mainFrame')[0].get('src')
    post = urllib.request.urlopen(blog_url).read()
    post_html = BeautifulSoup(post, 'html.parser')
    post_content_text = ""

    for post_content in post_html.find_all('div', {'class':'se-main-container'}):
        post_content_text += post_content.get_text()

    remove_break = re.compile('\n')
    post_content_text = re.sub(remove_break, '', post_content_text)

    return post_content_text

In [ ]:
import os
import sys
import urllib.request
import pandas as pd
import json
import re

client_id = "VzhDPPl2jYhHHqnnCnXG"
client_secret = "XD20x9HONl"

Qust_list = ['롯데 배송', '신세계 배송', 'ssg', 'ssg 배송']

for Q_query in Qust_list:
    query = urllib.parse.quote(Q_query)
    # encText = urllib.parse.quote("컴퓨터") # quote가 문자열을 utf-8으로 변경해줌
    idx = 0
    display = 100 # 가져올 정보의 개수 , default = 10, max = 100
    start = 1
    end = 1000
    sort = "sim"

    blog_df = pd.DataFrame(columns = ('Title', 'Link', 'Description', 'Blogger Name', 'Blogger Link', 'Post Date', 'Post Content'))

    for start_index in range(start, end, display):

        url = "https://openapi.naver.com/v1/search/blog?query=" + query \
        + "&display=" + str(display) \
        + "&start=" + str(start_index) \
        + "&sort=" + sort

        request = urllib.request.Request(url) 
        request.add_header("X-Naver-Client-Id",client_id)
        request.add_header("X-Naver-Client-Secret",client_secret)
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        if (rescode==200): # 정상코드
            response_body = response.read()
            response_dict = json.loads(response_body.decode('utf-8'))
            items = response_dict['items']
            for item_index in range(0, len(items)):
                remove_tag = re.compile('<.*?>')
                title = re.sub(remove_tag, '', items[item_index]['title'])
                link = items[item_index]['link']
                description = re.sub(remove_tag, '', items[item_index]['description'])
                blogger_name = items[item_index]['bloggername']
                blogger_link = items[item_index]['bloggerlink']
                post_data = items[item_index]['postdate']
                try:
                    post_content = get_posts(link)
                except:
                    pass

                blog_df.loc[idx] = [title, link, description, blogger_name, blogger_link, post_data, post_content]
                idx += 1
                if idx % 10 == 0:
                    print('#', end = "")
                
                if idx % 100 == 0:
                    print(f'{idx}번째 자료 수집 완료')       
        else:
            print("Error Code:" + rescode)


    blog_df.to_csv(f'blog_df_{Q_query}.csv', encoding='utf-8-sig')
    print(f'{Q_query} 수집 완료')



##########100번째 자료 수집 완료
##########200번째 자료 수집 완료
##########300번째 자료 수집 완료
##########400번째 자료 수집 완료
##########500번째 자료 수집 완료
##########600번째 자료 수집 완료
##########700번째 자료 수집 완료
##########800번째 자료 수집 완료
##########900번째 자료 수집 완료
##########1000번째 자료 수집 완료
롯데 배송 수집 완료
##########100번째 자료 수집 완료
##########200번째 자료 수집 완료
##########300번째 자료 수집 완료
##########400번째 자료 수집 완료
##########500번째 자료 수집 완료
##########600번째 자료 수집 완료
##########700번째 자료 수집 완료
##########800번째 자료 수집 완료
##########900번째 자료 수집 완료
##########1000번째 자료 수집 완료
신세계 배송 수집 완료
##########100번째 자료 수집 완료
##########200번째 자료 수집 완료
##########300번째 자료 수집 완료
##########400번째 자료 수집 완료
##########500번째 자료 수집 완료
##########600번째 자료 수집 완료
##########700번째 자료 수집 완료
##########800번째 자료 수집 완료
##########900번째 자료 수집 완료
##########1000번째 자료 수집 완료
ssg 수집 완료
#########